In [1]:


#Packages needed for testing
import snowflake.connector
import pickle
import boto3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#Pickle file for TFIDF
#Pickle file for Logistic Regression model results


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1604935257368_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user='APP_SPARK_PROD',
    password='>6txhcQ*7&^Qrq`j'
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  r.review_headline, r.review_comments, \
case when (contains(r.observations, ',NR,') or substring(r.observations, 1, 2) = 'NR')  then 1 else 0 end as NR \
from analytics.review as r \
where r.locale in ('en_US','en_GB','en_CA') \
and r.created_date > '2018-10-01' \
limit 1000000 "


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(100)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                 REVIEW_HEADLINE  \
0   227332653  I like the product. It appears good quality.     
1   227329614  Great pump excellent quality                     
2   227304633  Not Blue!                                        
3   227327655  HAPPY, HAPPY                                     
4   227328707  Excellent product                                
..        ...                ...                                
95  227321035  I would buy this item again                      
96  227309946  I would not buy this product again               
97  227322559  Works great and so quiet!                        
98  227329009  I bought one then after using I bought another   
99  227334893  Westinghouse WE55UJ4108 55 4K UHD Smart LED TV   

                                                                                                                                                                                                                           

In [35]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   UGC_ID           1000000 non-null  int64 
 1   REVIEW_HEADLINE  999922 non-null   object
 2   REVIEW_COMMENTS  998559 non-null   object
 3   NR               1000000 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 30.5+ MB

In [36]:
reviews['ALL_TEXT'] = 'headline ' + reviews['REVIEW_HEADLINE'] + ' comment ' + reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
reviews.head(100)

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                 REVIEW_HEADLINE  \
0   227332653  I like the product. It appears good quality.     
1   227329614  Great pump excellent quality                     
2   227304633  Not Blue!                                        
3   227327655  HAPPY, HAPPY                                     
4   227328707  Excellent product                                
..        ...                ...                                
95  227321035  I would buy this item again                      
96  227309946  I would not buy this product again               
97  227322559  Works great and so quiet!                        
98  227329009  I bought one then after using I bought another   
99  227334893  Westinghouse WE55UJ4108 55 4K UHD Smart LED TV   

                                                                                                                                                                                                                           

In [37]:
X = reviews[['ALL_TEXT']]
#X

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#Open pickle file from S3 bucket back into this notebook
s3 = boto3.resource('s3')
obj = s3.Object('pwr-analytics-emr','tfidf.pkl')
body = obj.get()['Body'].read()
tfidf = pickle.loads(body)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
#Use vocab from pickle file to transform text
tfidf_new = TfidfVectorizer(analyzer = 'word', max_features = 30000, stop_words= 'english', lowercase=True, vocabulary = tfidf.vocabulary_)
x_reviews = tfidf_new.fit_transform(X['ALL_TEXT'].apply(lambda x: np.str_(x)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
#Open pickle file for LR model
obj = s3.Object('pwr-analytics-emr','lr.pkl')
body = obj.get()['Body'].read()
lr = pickle.loads(body)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#Predict 0/1
y_pred = lr.predict(x_reviews)

#Predict probability
probs = lr.predict_proba(x_reviews)
y_probs = probs[:,1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
print(y_pred)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0 0 1 ... 0 0 0]

In [43]:
print(y_probs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.22866332 0.19255784 0.78817221 ... 0.02295464 0.01553197 0.07260956]